# Data Analysis Notebook for Benin

This notebook contains the setup and basic data analysis for **Benin**.

## Folder Structure
- data/Benin/raw
- data/Benin/processed
- notebooks/Benin


---


In [ ]:
# Import necessary libraries
import os
import pandas as pd

# Set up folder paths
base_dir = os.getcwd()
raw_data_dir = os.path.join(base_dir, 'data', 'Benin', 'raw')
processed_data_dir = os.path.join(base_dir, 'data', 'Benin', 'processed')

# Create directories if they don't exist
os.makedirs(raw_data_dir, exist_ok=True)
os.makedirs(processed_data_dir, exist_ok=True)

print(f"Folders created for Benin:")
print(f" - Raw data folder: {raw_data_dir}")
print(f" - Processed data folder: {processed_data_dir}")

# Example: Load a CSV file (replace 'your_file.csv' with actual file)
# file_path = os.path.join(raw_data_dir, 'your_file.csv')
# df = pd.read_csv(file_path)
# print(df.head())# List all cleaned files



[!] Raw file missing: data\Benin\raw\benin_raw.csv
